# Interactive vizualization of CaImAn `cnm.estmates` data

In [1]:
import pickle
import fastplotlib
import lazyarray
import mesmerize_core.arrays
import numpy as np
from skimage import io
from functools import partial

from fastplotlib import ImageWidget, Plot, GridPlot
from fastplotlib.graphics.line_slider import LineSlider
from ipywidgets import VBox, HBox, IntSlider, Layout, FloatRangeSlider

from caiman.utils.visualization import get_contours as caiman_get_contours

In [2]:
with open("Nikitas_mice/CNM_07_CA1_FS_2022_03_01_17_38_07_NR.pickle","rb") as f:
    cnm = pickle.load(f,)

In [3]:
def get_contours(cnm_object,kind="all"):
    contours = caiman_get_contours(cnm_object.A,dims=cnm_object.dims)
    coordinates = list()
    coms = list()

    for k,contour in enumerate(contours):
        if kind=="good":
            if k in cnm_object.idx_components_bad:
                continue
        if kind=="bad":
            if k in cnm_object.idx_components:
                continue
        coors = contour["coordinates"]
        coors = coors[~np.isnan(coors).any(axis=1)]
        coordinates.append(coors)

        com = coors.mean(axis=0)
        coms.append(com)

    return coordinates, coms

In [4]:
# Raw movie
raw = io.imread("mesmerize_root/Nikitas_mice/07_CA1_FS_2022_03_01_17_38_07_NR.tiff")

# Reconstructed movie
rcm = mesmerize_core.arrays.LazyArrayRCM(cnm.A, cnm.C, cnm.dims)

# Contours of spatial components (good)
contours, coms = get_contours(cnm) 
contours_good, coms_good = get_contours(cnm,"good") # Only good contours


# Temporal traces
temporal = cnm.C
temporal_good = cnm.C[cnm.idx_components]

ixs_good = cnm.idx_components
ixs_bad = cnm.idx_components_bad

In [5]:
rand_colors = np.random.rand(len(contours_good), 4)  # [n_contours, RGBA]
rand_colors[:, -1] = 1 # set alpha = 1

In [6]:
def euclidean(source, target, event, new_data):
    """maps click events to contour"""
    # calculate coms of line collection
    indices = np.array(event.pick_info["index"])
    print(event.pick_info)
    coms = list()

    for contour in target.graphics:
        coors = contour.data()[~np.isnan(contour.data()).any(axis=1)]
        com = coors.mean(axis=0)
        coms.append(com)

    # euclidean distance to find closest index of com 
    indices = np.append(indices, [0])
    
    ix = int(np.linalg.norm((coms - indices), axis=1).argsort()[0])
    
    target._set_feature(feature="colors", new_data=new_data, indices=ix)
    
    return None

## Full vizualization

In [14]:
cnmf_grid = GridPlot((1, 2),controllers="sync")
cnmf_grid.canvas.set_logical_size(1200,600)


raw_graphic = cnmf_grid[0,0].add_image(raw[0], cmap="binary_r")
rcm_graphic = cnmf_grid[0,1].add_image(rcm[0], cmap="magma")

rand_colors = np.random.rand(len(contours_good), 4)  # [n_contours, RGBA]
rand_colors[:, -1] = 1 


# add contours to both movie and rcm subplots
contours_raw = cnmf_grid[0,0].add_line_collection(contours_good, colors=rand_colors)
contours_rcm = cnmf_grid[0,1].add_line_collection(contours_good, colors="gray", thickness=1.0)

# --- Temporal stack

# temporal_stack_plot = fastplotlib.Plot()
# temporal_stack_graphic = temporal_stack_plot.add_line_stack(temporal_good, colors=rand_colors, thickness=1.0, separate=35)
#temporal_stack_plot.canvas.set_logical_size(400,400)

# --- Single temporal component
temporal_single_plot = fastplotlib.Plot()
temporal_single_graphic = temporal_single_plot.add_line_collection(temporal_good, colors=rand_colors) # Initially we display all temporal components
temporal_single_plot.canvas.set_logical_size(1200,150)


# Sliders:
slider = IntSlider(min=0, max=raw.shape[0] - 1, value=0, step=1)
clim_slider = FloatRangeSlider(min=rcm.min, max=rcm.max, step=0.1)
_ls = LineSlider(x_pos=0, bounds=(temporal.min(), temporal.max()), slider=slider)
#_ls2 = LineSlider(x_pos=0, bounds=(temporal.min(), temporal.max() + temporal_stack_plot.graphics[-1].position.y), slider=slider)
temporal_single_plot.add_graphic(_ls)
#temporal_stack_plot.add_graphic(_ls2)

# Functions to update each frame
def update_frame(change):
    ix = change["new"]
    raw_graphic.data = raw[ix]
    rcm_graphic.data = rcm[ix]

def update_clims(change):
    vmin, vmax = change["new"]
    rcm_graphic.vmax=vmax
    rcm_graphic.vmin=vmin
    
slider.observe(update_frame, "value")
clim_slider.observe(update_clims, "value")
    
    
@temporal_single_plot.renderer.add_event_handler("resize")
def update_slider_width(*args):
    width, h = temporal_single_plot.renderer.logical_size
    slider.layout = Layout(width=f"{width}px")


#VBox([temporal_single_plot.show(), HBox([cnmf_grid.show(),temporal_stack_plot.show()]) , slider,clim_slider])
VBox([temporal_single_plot.show(),cnmf_grid.show(), slider, clim_slider])

RFBOutputContext()

RFBOutputContext()

## Interactivity

Evaluate afer the cell above

In [15]:
# so we can view them one by one, first hide all of them
temporal_single_graphic[:].present = False

# link image to contours
raw_graphic.link(
    "click",
    target=contours_raw,
    feature="colors", 
    new_data="w", 
    callback=euclidean 
)

# link image to contours
rcm_graphic.link(
    "click",
    target=contours_rcm,
    feature="colors", 
    new_data="w", 
    callback=euclidean 
)

# contours colors -> contor thickness
contours_raw.link("colors", target=contours_raw, feature="thickness", new_data=2.)
contours_rcm.link("colors", target=contours_rcm, feature="thickness", new_data=2.)

# contours_movie <-> contours_rcm
contours_rcm.link("colors", target=contours_raw, feature="colors", new_data="w", bidirectional=True)

# # temporal stack events
# temporal_stack_graphic.link("click", target=temporal_stack_graphic, feature="colors", new_data="w")
# temporal_stack_graphic.link("colors", target=temporal_stack_graphic, feature="thickness", new_data=4)

# contours <-> temporal stack
contours_raw.link("colors", target=temporal_single_graphic, feature="present", new_data=True)

# temporal stack -> temporal single
#temporal_stack_graphic.link("colors", target=temporal_single_graphic, feature="present", new_data=True)

# autoscale temporal plot to the current temporal component
temporal_single_graphic[:].present.add_event_handler(temporal_single_plot.auto_scale)

/Users/artemkirsanov/opt/anaconda3/envs/caiman/lib/python3.10/site-packages/fastplotlib/graphics/features/_base.py:87: UserWarning: Event handler <bound method Interaction.event_handler of LineCollection @ 0x35fca1300
Collection of <15> Graphics> is already registered.
  warn(f"Event handler {handler} is already registered.")
/Users/artemkirsanov/opt/anaconda3/envs/caiman/lib/python3.10/site-packages/fastplotlib/graphics/features/_base.py:87: UserWarning: Event handler <bound method Interaction.event_handler of LineCollection @ 0x35fca0700
Collection of <15> Graphics> is already registered.
  warn(f"Event handler {handler} is already registered.")
